In [1]:
import numpy as np
from copy import deepcopy

In [2]:
filename = "../../data/gcode/dino_body_sliced_perimeter_speed_120.gcode"
f = open(filename)
x = f.readlines()

In [3]:
def cut_header(x):
    '''
    Returns index for beginning of linear commands.
    NOTE: this applies to slic3r + taz6 boilerplate format. heuristic: linear commands start with 3rd "G92 E0".
    '''
    counter = 0
    for i,line in enumerate(x):
        split_line = line.split("\n")
        if str.startswith(split_line[0], "G92 E0"):
            counter = counter + 1
            if counter == 3: # if third instance
                return i

def cut_footer(x):
    '''
    Return index for end of linear commands.
    NOTE: this applies to slic3r + taz6 boilerplate format. heuristic: find index for second M400 command.
    '''
    counter = 0
    for i,line in enumerate(x):
        split_line = line.split("\n")
        if str.startswith(split_line[0], "M400"):
            counter = counter + 1
            if counter == 2: # if third instance
                return i-1

def gcode_to_training_data(x, training_data):
    cmd_to_feature = {'G': 0, 'X': 1, 'Y': 2, 'Z': 3, 'E': 4, 'F': 5}
    curr_values = [0,0,0,0,0,0]
    for row,line in enumerate(x):
        sep_line = line.split("\n")[0].split("\n")[0].split(" ")
        line_cmds = []
        for cmd in sep_line:
            col = cmd_to_feature[cmd[0]]
            value = cmd[1:]
            curr_values[col] = value
        training_data[row] = curr_values
    return training_data

In [4]:
def normalize_training_data(x):
    # convert G1 to +1 and G92 to -1
    x[:, 0][x[:, 0] == 92] = -1
    # normalize columns 1 to end by dividing by max
    x[:, 1:] = x[:, 1:] / np.max(x[:, 1:],axis=0)
    return x

In [5]:
start_ind = cut_header(x)
end_ind = cut_footer(x)
num_commands = end_ind - start_ind

training_data = np.zeros((num_commands, 6))
x_trimmed = x[start_ind: end_ind]

train_raw = gcode_to_training_data(x_trimmed, training_data)

In [6]:
train = normalize_training_data(deepcopy(train_raw))

In [7]:
train_raw[0:10]

array([[9.20000e+01, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00],
       [1.00000e+00, 0.00000e+00, 0.00000e+00, 3.00000e-01, 0.00000e+00,
        1.08000e+04],
       [1.00000e+00, 7.84960e+01, 6.76660e+01, 3.00000e-01, 0.00000e+00,
        1.08000e+04],
       [1.00000e+00, 7.84960e+01, 6.76660e+01, 3.00000e-01, 2.00000e+00,
        1.50000e+03],
       [1.00000e+00, 8.01280e+01, 6.62690e+01, 3.00000e-01, 2.04280e+00,
        1.80000e+03],
       [1.00000e+00, 8.24630e+01, 6.48330e+01, 3.00000e-01, 2.09741e+00,
        1.80000e+03],
       [1.00000e+00, 8.33960e+01, 6.43540e+01, 3.00000e-01, 2.11830e+00,
        1.80000e+03],
       [1.00000e+00, 8.44090e+01, 6.39280e+01, 3.00000e-01, 2.14021e+00,
        1.80000e+03],
       [1.00000e+00, 8.75670e+01, 6.34070e+01, 3.00000e-01, 2.20396e+00,
        1.80000e+03],
       [1.00000e+00, 1.13568e+02, 6.52750e+01, 3.00000e-01, 2.72331e+00,
        1.80000e+03]])

In [8]:
train[0:10]

array([[-1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 1.        ,  0.        ,  0.        ,  0.01062323,  0.        ,
         1.        ],
       [ 1.        ,  0.62860163,  0.49399895,  0.01062323,  0.        ,
         1.        ],
       [ 1.        ,  0.62860163,  0.49399895,  0.01062323,  0.04055239,
         0.13888889],
       [ 1.        ,  0.6416708 ,  0.48380008,  0.01062323,  0.04142021,
         0.16666667],
       [ 1.        ,  0.66036965,  0.47331649,  0.01062323,  0.04252749,
         0.16666667],
       [ 1.        ,  0.66784118,  0.46981953,  0.01062323,  0.04295106,
         0.16666667],
       [ 1.        ,  0.67595336,  0.4667095 ,  0.01062323,  0.04339531,
         0.16666667],
       [ 1.        ,  0.70124285,  0.46290591,  0.01062323,  0.04468792,
         0.16666667],
       [ 1.        ,  0.90946074,  0.47654334,  0.01062323,  0.05521836,
         0.16666667]])

In [ ]:
train_raw[0:10]

In [ ]:
plt.figure(figsize=(20,4))
plt.plot(train[0:175,1])

In [ ]:
plt.figure(figsize=(20,4))
plt.plot(train[0:500,1])

In [ ]:
plt.figure(figsize=(20,4))
plt.plot(train[0:2000,1])

In [ ]:
plt.plot(train[0:100,1])
plt.plot(train[0:500,1])
plt.plot(train[0:1000,1])

In [ ]:
plt.plot(train[0:1000,2])